In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np


In [2]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [3]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [4]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [5]:
def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [6]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [7]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


In [8]:
import webbrowser

In [9]:
import tkinter
from tkinter import *
import webbrowser

y=0
l=66



def adjust(s):
    l = len(s)
    line = l//38
    line = line+1
    for i in range(1,line):
        s = s[:37*i]+'\n'+s[37*i:]
    return s,line

def send():
    
    def go():
        s = txt.get()
        webbrowser.open_new(f"https://www.google.com/maps/search/hospital+in+{s}")
        
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        global y
        global l
        
        y=y+l
        msg,length = adjust(msg)
        print(length)
        lb = Label(base,text=msg, bg="LightBlue3",font="15",justify="left")
        c.create_window(360,y,window=lb,anchor="ne")
    
        res = chatbot_response(msg)
        
            
        l=33*length
        y = y+l
        line = len(res)
        line = (line//43) +1
        lb2 =Label(base,text=res, bg="DarkOliveGreen3",font="15",justify="left")
        c.create_window(10,y,window=lb2,anchor="nw")
        print(line)
        l=33*line
        if res == "Adverse drug": 
            webbrowser.open_new("https://www.aafp.org/pubs/afp/issues/2003/1101/p1781.html")
        if res == "hospital" or res=="pharmacy":
            newWindow = Toplevel(base)
            lab = Label(newWindow,text="Enter your location: ").pack()
            txt = Entry(newWindow,width = 20)
            txt.pack()
            btn = Button(newWindow,text="ok",command=go).pack()
            

base = Tk()
base.title("Hello")
base.geometry("400x515")
base.resizable(width=FALSE, height=TRUE)

frame = Frame(base,width=400,height=500)
frame.pack(expand=True, fill=BOTH)
c = Canvas(frame, height="400", width="400",bg ="#F0F0F8",scrollregion=(0,0,0,7000))
vertibar=Scrollbar(frame,orient=VERTICAL)
vertibar.pack(side=RIGHT,fill=Y)
vertibar.config(command=c.yview)
c.config(width=400,height=400)
c.config(yscrollcommand=vertibar.set)
c.pack(expand=True,side=LEFT,fill=BOTH)
lb = Label(c,text="hi. this is your personal health care support.\nAsk what you need to know.", bg="DarkOliveGreen3",font="15",justify="left")
c.create_window(10,10,window=lb,anchor="nw")
frame = Frame(base,width=400,height=500)
frame.pack(expand=True, fill=BOTH)
EntryBox = Text(frame, bd=0, bg="white",width="80", height="1", font="Arial",borderwidth=1, relief="solid")
SendButton = Button(frame, font=("Verdana",12,'bold'), text="Send", width="6", height="1",
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
EntryBox.place(x=6, y=30, height=70, width=300)
SendButton.place(x=310, y=30, height=30)

base.mainloop()


1
1/1 [==============================] - 0s 334ms/step
1
1
1/1 [==============================] - 0s 25ms/step
3
1
1/1 [==============================] - 0s 27ms/step
1
1
1/1 [==============================] - 0s 26ms/step
1


In [10]:
'''
import tkinter
from tkinter import *

def chatlogs():
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "Bot: Hi. I am Chatbot. Your Health care Partner. Ask What u want to know.\n\n")
    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265",font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

chatlogs()

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="10", font="Arial")
#EntryBox.bind("<Return>", send)

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=6, y=401, height=90, width=265)
SendButton.place(x=268, y=401, height=90)

base.mainloop()


'''


'\nimport tkinter\nfrom tkinter import *\n\ndef chatlogs():\n    ChatLog.config(state=NORMAL)\n    ChatLog.insert(END, "Bot: Hi. I am Chatbot. Your Health care Partner. Ask What u want to know.\n\n")\n    ChatLog.config(state=DISABLED)\n    ChatLog.yview(END)\n\n\ndef send():\n    msg = EntryBox.get("1.0",\'end-1c\').strip()\n    EntryBox.delete("0.0",END)\n\n    if msg != \'\':\n        ChatLog.config(state=NORMAL)\n        ChatLog.insert(END, "You: " + msg + \'\n\n\')\n        ChatLog.config(foreground="#442265",font=("Verdana", 12 ))\n    \n        res = chatbot_response(msg)\n        ChatLog.insert(END, "Bot: " + res + \'\n\n\')\n            \n        ChatLog.config(state=DISABLED)\n        ChatLog.yview(END)\n\nbase = Tk()\nbase.title("Hello")\nbase.geometry("400x500")\nbase.resizable(width=FALSE, height=FALSE)\n\n#Create Chat window\nChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)\n\nChatLog.config(state=DISABLED)\n\nchatlogs()\n\n#Bind scrollbar to 

In [11]:
'''
while True:
    msg = input("Enter dialog:")
    if msg != '':
        res = chatbot_response(msg)
        print(res)
'''


'\nwhile True:\n    msg = input("Enter dialog:")\n    if msg != \'\':\n        res = chatbot_response(msg)\n        print(res)\n'

In [12]:
'''  
from tkinter import *
import webbrowser
window=Tk()
# add widgets here
window.title('Chat Bot')
window.geometry("600x400+10+20")

def go():
    loc = entry2.get()
    webbrowser.open_new(f"https://www.google.com/maps/search/hospital+in+{loc}")
    label2.pack_forget()
    entry2.pack_forget()
    b2.pack_forget()

def cmde(e):
    cmd()

def cmd():
    msg = entry1.get()
    if msg != '':
        res = chatbot_response(msg)
        entry1.delete(0,"end")
        label.configure(text=res)
    if res == "Adverse drug": 
        webbrowser.open_new("https://www.aafp.org/pubs/afp/issues/2003/1101/p1781.html")
    if res == "hospital" or res=="pharmacy":
        print("hi")
        label2.pack()
        entry2.pack()
        b2.pack()
        

        
label0 =Label(window,text="Welcome to\nAutomated Health Care Assistant",font = ("Arial", 20)).pack(padx=10,pady=20)
entry1 = Entry(window,width=60)
entry1.pack(padx=10,pady=10)
window.bind('<Return>',cmde)
Button(window,text="Enter",command=cmd).pack()
label = Label(window,text="")
label.pack()
label2 = Label(window,text="Enter your location: ")
entry2 = Entry(window,width = 20)
b2 = Button(window,text="Enter",command=go)
window.mainloop()


'''

'  \nfrom tkinter import *\nimport webbrowser\nwindow=Tk()\n# add widgets here\nwindow.title(\'Chat Bot\')\nwindow.geometry("600x400+10+20")\n\ndef go():\n    loc = entry2.get()\n    webbrowser.open_new(f"https://www.google.com/maps/search/hospital+in+{loc}")\n    label2.pack_forget()\n    entry2.pack_forget()\n    b2.pack_forget()\n\ndef cmde(e):\n    cmd()\n\ndef cmd():\n    msg = entry1.get()\n    if msg != \'\':\n        res = chatbot_response(msg)\n        entry1.delete(0,"end")\n        label.configure(text=res)\n    if res == "Adverse drug": \n        webbrowser.open_new("https://www.aafp.org/pubs/afp/issues/2003/1101/p1781.html")\n    if res == "hospital" or res=="pharmacy":\n        print("hi")\n        label2.pack()\n        entry2.pack()\n        b2.pack()\n        \n\n        \nlabel0 =Label(window,text="Welcome to\nAutomated Health Care Assistant",font = ("Arial", 20)).pack(padx=10,pady=20)\nentry1 = Entry(window,width=60)\nentry1.pack(padx=10,pady=10)\nwindow.bind(\'<Retu

In [13]:
"""
import webbrowser

while True:
    msg = input("inter dialog: ")
    if msg != '':
        res = chatbot_response(msg)
        print(res)
        if res == "Adverse drug": 
            webbrowser.open_new("https://www.aafp.org/pubs/afp/issues/2003/1101/p1781.html")
        if res == "hospital":
            loc = input("enter city name: ")
            webbrowser.open_new(f"https://www.google.com/maps/search/hospital+in+{loc}")
        if res == "pharmacy":
            loc = input("enter city name: ")
            webbrowser.open_new(f"https://www.google.com/maps/search/hospital+in+{loc}")
"""


'\nimport webbrowser\n\nwhile True:\n    msg = input("inter dialog: ")\n    if msg != \'\':\n        res = chatbot_response(msg)\n        print(res)\n        if res == "Adverse drug": \n            webbrowser.open_new("https://www.aafp.org/pubs/afp/issues/2003/1101/p1781.html")\n        if res == "hospital":\n            loc = input("enter city name: ")\n            webbrowser.open_new(f"https://www.google.com/maps/search/hospital+in+{loc}")\n        if res == "pharmacy":\n            loc = input("enter city name: ")\n            webbrowser.open_new(f"https://www.google.com/maps/search/hospital+in+{loc}")\n'